In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 10.6 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=9d79ce1cc62075d154099a6b26e535fab268c328a2cb01c739a26026b2db2978
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  At

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import lightgbm as lgb
import datetime as dt
import optuna
import gc
import warnings
import random
import os
from functools import partial
from scipy import stats

warnings.simplefilter('ignore')

In [ ]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
fix_all_seeds(0)

# データ読み込み

In [ ]:
df_train_val = pd.read_pickle("/content/drive/MyDrive/Kaggle/花粉飛散量（ProbSpace）/proc/20221210/val24.pkl")
df_train_val.dropna(inplace=True)
df_train_val.head()

,datetime,precipitation_utsunomiya,precipitation_chiba,precipitation_tokyo,temperature_utsunomiya,temperature_chiba,temperature_tokyo,winddirection_utsunomiya,winddirection_chiba,winddirection_tokyo,...,mean_temperature,mean_cumsum_temperature,mean_windspeed,mean_zerocount_precipitation,utsunomiya_rising,utsunomiya_setting,seconds_since_rising,temp_transition_utsunomiya,temp_transition_tokyo,temp_transition_chiba
datetime2,,,,,,,,,,,,,,,,,,,,,
2017-02-08 00:00:00,2017020724,0.0,0.0,0.0,-0.5,4.0,3.2,6,14.0,15.0,...,2.233333,40.581944,3.466667,4.342150,2017-02-08 06:35:52.904637,2017-02-08 17:13:48.727028,-23753.0,0.1,0.0,0.0
2017-02-08 01:00:00,2017020801,0.0,0.0,0.0,-1.1,3.9,2.9,5,14.0,14.0,...,1.900000,45.579167,4.133333,4.356890,2017-02-08 06:35:52.904637,2017-02-08 17:13:48.727028,-20153.0,0.0,0.0,0.0
2017-02-08 02:00:00,2017020802,0.0,0.0,0.0,-1.9,3.8,1.7,3,14.0,1.0,...,1.200000,45.579167,2.966667,4.371377,2017-02-08 06:35:52.904637,2017-02-08 17:13:48.727028,-16553.0,0.0,0.0,0.0
2017-02-08 03:00:00,2017020803,0.0,0.0,0.0,-2.4,3.7,2.6,7,14.0,13.0,...,1.300000,45.579167,2.700000,4.385619,2017-02-08 06:35:52.904637,2017-02-08 17:13:48.727028,-12953.0,0.0,0.9,0.0
2017-02-08 04:00:00,2017020804,0.0,0.0,0.0,-2.4,3.8,2.8,3,14.0,14.0,...,1.400000,45.579167,2.733333,4.399627,2017-02-08 06:35:52.904637,2017-02-08 17:13:48.727028,-9353.0,0.0,1.1,0.1


# 特徴量

In [ ]:
features = ['std18hours_windspeed_utsunomiya',
 'MSUM12hours_log_precipitation_tokyo',
 'log_precipitation_tokyo',
 'log_precipitation_chiba',
 'windspeed_chiba_EW',
 'MA7days_temperature_utsunomiya',
 'MSUM24hours_log_precipitation_chiba',
 'std18hours_windspeed_chiba_NS',
 'diffMA3_cumsum_temperature_utsunomiya',
 'windspeed_tokyo_NS',
 'MA18hours_windspeed_chiba',
 'zerocount_precipitation_chiba',
 'hour',
 'MA18hours_windspeed_utsunomiya_NS',
 'mean_zerocount_precipitation',
 'MA18hours_windspeed_utsunomiya_EW',
 'std18hours_windspeed_chiba_EW',
 'temperature_chiba',
 'std18hours_windspeed_utsunomiya_EW',
 'mean_log_precipitation',
 'shiftRatio_diffMA3_cumsum_temperature_utsunomiya',
 'maxmin_temperature_utsunomiya',
 'MSUM48hours_log_precipitation_tokyo',
 'MSUM12hours_log_precipitation_chiba',
 'MSUM12hours_log_precipitation_utsunomiya',
 'maxmin_temperature_chiba',
 'minmax_temperature_utsunomiya',
 'windspeed_utsunomiya_EW',
 'windspeed_utsunomiya',
 'MA7days_temperature_tokyo',
 'MA18hours_windspeed_tokyo_NS',
 'minmax_temperature_chiba',
 'MA18hours_windspeed_utsunomiya',
 'mean_windspeed',
 'maxmin_temperature_tokyo',
 'seconds_since_rising',
 'MA18hours_windspeed_tokyo',
 'mean_cumsum_temperature',
 'zerocount_precipitation_tokyo',
 'windspeed_utsunomiya_NS',
 'MA18hours_windspeed_chiba_EW',
 'std18hours_windspeed_tokyo',
 'zerocount_precipitation_utsunomiya',
 'MA18hours_windspeed_tokyo_EW',
 'diff1_cumsum_pollen_ratio_fitted_chiba',
 'shiftRatio_diffMA3_cumsum_temperature_chiba',
 'windspeed_tokyo',
 'minmax_temperature_tokyo',
 'MSUM24hours_log_precipitation_tokyo',
 'MA7days_temperature_chiba',
 'std18hours_windspeed_tokyo_EW',
 'std18hours_windspeed_utsunomiya_NS',
 'windspeed_chiba_NS',
 'MSUM48hours_log_precipitation_utsunomiya',
 'MSUM48hours_log_precipitation_chiba',
 'log_precipitation_utsunomiya',
 'std18hours_windspeed_tokyo_NS',
 'MA18hours_windspeed_chiba_NS',
 'MSUM24hours_log_precipitation_utsunomiya',
 'std18hours_windspeed_chiba',
 'windspeed_chiba',
 'shiftRatio_diffMA3_cumsum_temperature_tokyo',
 'windspeed_tokyo_EW']

print("特徴量の数：", len(features))

特徴量の数： 63


# Optuna

In [ ]:
def my_kfold(df):
    idx = df.index
    cv = []
    train_idx1 = (idx < dt.datetime(2018, 4, 1))
    val_idx1 = (idx >= dt.datetime(2018, 4, 1))&\
               (idx < dt.datetime(2018, 4, 15))
    cv.append([train_idx1, val_idx1])
    train_idx2 = (idx < dt.datetime(2018, 4, 15))
    val_idx2 = (idx >= dt.datetime(2018, 4, 15))&\
               (idx < dt.datetime(2018, 4, 29))
    cv.append([train_idx2, val_idx2])
    train_idx3 = (idx < dt.datetime(2019, 4, 1))
    val_idx3 = (idx >= dt.datetime(2019, 4, 1))&\
               (idx < dt.datetime(2019, 4, 15))
    cv.append([train_idx3, val_idx3])
    train_idx4 = (idx < dt.datetime(2019, 4, 15))
    val_idx4 = (idx >= dt.datetime(2019, 4, 15))&\
               (idx < dt.datetime(2019, 4, 29))
    cv.append([train_idx4, val_idx4])
    return cv

In [ ]:
def objective(X, y, y_raw, cv_indicies, trial):

    # Optuna suggest params
    params = {
        # "extra_trees": True,
        'objective': 'fair',
        'metric': 'fair',
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 3, 27),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.002, 0.10),
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.05, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.05, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 5, 256),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3, 0.8),
        # 'subsample': trial.suggest_uniform('subsample', 0.4, 0.9),
#         'subsample_freq': trial.suggest_int('subsample_freq', 1, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        "verbose": -1,
        'seed': 46,
        "n_jobs": -1
    }

    # FN_ratio = trial.suggest_uniform('FN_ratio', 0, 1)
    
    scores = []
    for train_idx, val_idx in cv_indicies:
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_val = X[val_idx]
        y_val = y[val_idx]
        y_val_raw = y_raw[val_idx]
       
        train_dataset = lgb.Dataset(X_train, y_train, feature_name=features)
        
        model = lgb.train(params, train_dataset)
        y_pred = model.predict(X_val)
        y_pred = np.expm1(y_pred)
        y_pred[y_pred<0] = 0
        y_pred = np.round(y_pred) * 4

        mae = np.mean(np.abs(y_val_raw - y_pred))
        scores.append(mae)
    print("Scores:", scores)
    return np.average(scores)

# 宇都宮

In [ ]:
cv_indicies = my_kfold(df_train_val)

train_val_X = df_train_val[features].values
train_val_y = df_train_val["log_pollen_utsunomiya"].values
train_val_y_raw = df_train_val["pollen_utsunomiya"].values
# S_code = price_features[["Code"]]

# del df_train_val
# gc.collect()

In [ ]:
f = partial(objective,
            train_val_X,
            train_val_y,
            train_val_y_raw,
            cv_indicies)
study = optuna.create_study(direction="minimize")
study.optimize(f, n_trials=100)

trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
best_params = trial.params  

[I 2023-01-08 22:41:58,480] A new study created in memory with name: no-name-5ec972b0-e596-4b21-8a8e-a1dcce952629
[I 2023-01-08 22:44:19,097] Trial 0 finished with value: 11.175204387111618 and parameters: {'max_depth': 27, 'n_estimators': 1977, 'learning_rate': 0.06778984774717976, 'lambda_l2': 0.33841507645435304, 'num_leaves': 252, 'colsample_bytree': 0.36309312940708416}. Best is trial 0 with value: 11.175204387111618.


Scores: [16.08900671125707, 5.480742849154305, 15.945257054295986, 7.185810933739114]


[I 2023-01-08 22:44:27,445] Trial 1 finished with value: 10.780429138285132 and parameters: {'max_depth': 8, 'n_estimators': 371, 'learning_rate': 0.027549967818334305, 'lambda_l2': 5.474191036388837, 'num_leaves': 20, 'colsample_bytree': 0.7759477304725235}. Best is trial 1 with value: 10.780429138285132.


Scores: [15.038613993697144, 5.353272472091476, 15.670440762363686, 7.059389324988228]


[I 2023-01-08 22:45:17,644] Trial 2 finished with value: 11.347468782865185 and parameters: {'max_depth': 5, 'n_estimators': 2943, 'learning_rate': 0.06078280206588705, 'lambda_l2': 0.6120252988199267, 'num_leaves': 211, 'colsample_bytree': 0.7720504852245931}. Best is trial 1 with value: 10.780429138285132.


...

Scores: [14.904062422483404, 5.391088662295581, 15.334520509069353, 7.0812910570754]


[I 2023-01-08 23:24:27,371] Trial 98 finished with value: 10.67852852665159 and parameters: {'max_depth': 3, 'n_estimators': 339, 'learning_rate': 0.07932680932682425, 'lambda_l2': 4.3718909818707, 'num_leaves': 219, 'colsample_bytree': 0.7465679908187243}. Best is trial 83 with value: 10.459988904496717.


Scores: [14.991612737814492, 4.918291316696895, 15.557686586394114, 7.246523465700862]


[I 2023-01-08 23:24:43,084] Trial 99 finished with value: 10.57020168972314 and parameters: {'max_depth': 5, 'n_estimators': 750, 'learning_rate': 0.005823617087407098, 'lambda_l2': 5.352467384966575, 'num_leaves': 205, 'colsample_bytree': 0.797681012290383}. Best is trial 83 with value: 10.459988904496717.


Scores: [14.533226906044858, 5.121723221322758, 15.250100466650945, 7.375756164873996]
  Value: 10.459988904496717
  Params: 
    max_depth: 3
    n_estimators: 866
    learning_rate: 0.01397557995522496
    lambda_l2: 4.611316892731575
    num_leaves: 167
    colsample_bytree: 0.41815543878285094


In [ ]:
best_params

{'max_depth': 3,
 'n_estimators': 866,
 'learning_rate': 0.01397557995522496,
 'lambda_l2': 4.611316892731575,
 'num_leaves': 167,
 'colsample_bytree': 0.41815543878285094}

# 東京

In [ ]:
train_val_y = df_train_val["log_pollen_tokyo"].values
train_val_y_raw = df_train_val["pollen_tokyo"].values

f = partial(objective,
            train_val_X,
            train_val_y,
            train_val_y_raw,
            cv_indicies)
study = optuna.create_study(direction="minimize")
study.optimize(f, n_trials=100)

trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
best_params = trial.params

[I 2023-01-08 23:24:43,123] A new study created in memory with name: no-name-984a9f9d-4e4a-4cf8-940a-471c3a1c9895
[I 2023-01-08 23:25:08,245] Trial 0 finished with value: 6.578617314250896 and parameters: {'max_depth': 6, 'n_estimators': 1179, 'learning_rate': 0.0877953639835753, 'lambda_l2': 0.5702320403194425, 'num_leaves': 41, 'colsample_bytree': 0.7537787957083981}. Best is trial 0 with value: 6.578617314250896.


Scores: [11.96680301887801, 4.4873391086767995, 5.094673910784909, 4.765653218663864]


[I 2023-01-08 23:27:08,344] Trial 1 finished with value: 6.623788198911243 and parameters: {'max_depth': 25, 'n_estimators': 1033, 'learning_rate': 0.036320876302840616, 'lambda_l2': 6.311700754081432, 'num_leaves': 229, 'colsample_bytree': 0.728794395208836}. Best is trial 0 with value: 6.578617314250896.


Scores: [12.215653178544498, 4.6524302782337585, 4.903456778747881, 4.723612560118833]


[I 2023-01-08 23:27:10,898] Trial 2 finished with value: 6.427087173369171 and parameters: {'max_depth': 5, 'n_estimators': 297, 'learning_rate': 0.08250738991206417, 'lambda_l2': 3.7976457280359512, 'num_leaves': 59, 'colsample_bytree': 0.3004012711651385}. Best is trial 2 with value: 6.427087173369171.


...

Scores: [11.899300215844434, 4.643854231161746, 4.716197765542394, 4.816342403336678]


[I 2023-01-09 00:17:22,167] Trial 98 finished with value: 6.26305543194967 and parameters: {'max_depth': 23, 'n_estimators': 319, 'learning_rate': 0.006716736787706921, 'lambda_l2': 4.016773416780573, 'num_leaves': 84, 'colsample_bytree': 0.4354008364210428}. Best is trial 92 with value: 6.126969811052801.


Scores: [11.863662469506956, 3.748951667627721, 4.938973321083679, 4.500634269580326]


[I 2023-01-09 00:17:32,428] Trial 99 finished with value: 6.408122524584444 and parameters: {'max_depth': 14, 'n_estimators': 457, 'learning_rate': 0.05339884078422095, 'lambda_l2': 2.6822682829976174, 'num_leaves': 46, 'colsample_bytree': 0.3996297038414585}. Best is trial 92 with value: 6.126969811052801.


Scores: [12.070483707365765, 4.276223360092347, 4.952766586396865, 4.333016444482796]
  Value: 6.126969811052801
  Params: 
    max_depth: 23
    n_estimators: 160
    learning_rate: 0.009148271396819509
    lambda_l2: 2.4056285524139014
    num_leaves: 68
    colsample_bytree: 0.37313117665127893


In [ ]:
best_params

{'max_depth': 23,
 'n_estimators': 160,
 'learning_rate': 0.009148271396819509,
 'lambda_l2': 2.4056285524139014,
 'num_leaves': 68,
 'colsample_bytree': 0.37313117665127893}

# 千葉

In [ ]:
train_val_y = df_train_val["log_pollen_chiba"].values
train_val_y_raw = df_train_val["pollen_chiba"].values

f = partial(objective,
            train_val_X,
            train_val_y,
            train_val_y_raw,
            cv_indicies)
study = optuna.create_study(direction="minimize")
study.optimize(f, n_trials=100)

trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
best_params = trial.params

[I 2023-01-09 00:17:32,470] A new study created in memory with name: no-name-887c2377-e801-40c3-9445-ab7a1c06d8b0
[I 2023-01-09 00:18:31,496] Trial 0 finished with value: 7.290138019829341 and parameters: {'max_depth': 26, 'n_estimators': 991, 'learning_rate': 0.04593165844090655, 'lambda_l2': 0.16378872149733661, 'num_leaves': 129, 'colsample_bytree': 0.464781491337246}. Best is trial 0 with value: 7.290138019829341.


Scores: [13.816340125549935, 4.08246040567456, 6.233982273937156, 5.0277692741557125]


[I 2023-01-09 00:19:33,557] Trial 1 finished with value: 7.27787185318432 and parameters: {'max_depth': 19, 'n_estimators': 2051, 'learning_rate': 0.08104475967297785, 'lambda_l2': 0.9389285261872059, 'num_leaves': 48, 'colsample_bytree': 0.6256066071262791}. Best is trial 1 with value: 7.27787185318432.


Scores: [13.500629959184561, 4.385945632602603, 6.446402270380888, 4.77850955056923]


[I 2023-01-09 00:20:32,872] Trial 2 finished with value: 7.517487900095559 and parameters: {'max_depth': 10, 'n_estimators': 1559, 'learning_rate': 0.032097887412316416, 'lambda_l2': 1.5848001300362429, 'num_leaves': 54, 'colsample_bytree': 0.7993941102422277}. Best is trial 1 with value: 7.27787185318432.


...

Scores: [15.399112604123253, 3.8577868492829763, 5.741247918122478, 4.974255553168854]


[I 2023-01-09 01:16:52,718] Trial 98 finished with value: 7.379993946916569 and parameters: {'max_depth': 5, 'n_estimators': 1201, 'learning_rate': 0.03207682704121881, 'lambda_l2': 0.5033824162257245, 'num_leaves': 130, 'colsample_bytree': 0.787022129305033}. Best is trial 95 with value: 6.766457364343425.


Scores: [14.904581912185666, 3.807517945980552, 5.880499147536731, 4.927376781963323]


[I 2023-01-09 01:17:04,734] Trial 99 finished with value: 6.82266386207849 and parameters: {'max_depth': 3, 'n_estimators': 1612, 'learning_rate': 0.024220854290188255, 'lambda_l2': 0.6940099051751804, 'num_leaves': 104, 'colsample_bytree': 0.6882306077439566}. Best is trial 95 with value: 6.766457364343425.


Scores: [12.909283252419257, 3.8311304374623387, 5.365783350061637, 5.184458408370726]
  Value: 6.766457364343425
  Params: 
    max_depth: 3
    n_estimators: 1233
    learning_rate: 0.030139062254810847
    lambda_l2: 0.69825297208385
    num_leaves: 108
    colsample_bytree: 0.7706021026655606


In [ ]:
best_params

{'max_depth': 3,
 'n_estimators': 1233,
 'learning_rate': 0.030139062254810847,
 'lambda_l2': 0.69825297208385,
 'num_leaves': 108,
 'colsample_bytree': 0.7706021026655606}